In [1]:
from Bio import SeqIO
import collections
import pandas as pd 
import re
import requests
import json
import time
import collections
import glob
import pprint
import seaborn as sns
import pylab as plt
from matplotlib.ticker import MaxNLocator
from matplotlib.colors import ListedColormap
import pycountry 
import pycountry_convert as pc
pp = pprint.PrettyPrinter()
import numpy as np

## Transform publication-patent edges into CoR-CoP edges

In [3]:
# open the raw patent data again so that I can edit it
df_CoP = pd.read_csv('PATH\data\CoP_fractional_scores.csv')
df_CoR = pd.read_csv('PATH\data\CoR_fractional_scores.csv')

In [4]:
df_pub_patent_edges_withdup = pd.read_csv('PATH\data\pub_patent_edges_Genbank[OBIS_and_patents].csv')
df_pub_patent_edges = df_pub_patent_edges_withdup.drop_duplicates()

In [5]:
# merge patents onto df_sci
df_CoR_pat = pd.merge(df_CoR.drop(columns = {'Unnamed: 0', 'times_cited_sci', 'times_cited_pat', 'times_cited_sci_fract', 'times_cited_pat_fract'}), df_pub_patent_edges[['pubmed_IDs', 'patentID']].rename(columns = {'pubmed_IDs':'pubmed_id'}), on = 'pubmed_id')

In [6]:
df_CoR_CoP = pd.merge(df_CoR_pat, df_CoP.rename(columns = {'patent_id': 'patentID'}).drop(columns = {'patent_citations_fract', 'patent_citations', 'Unnamed: 0'}), on = 'patentID').rename(columns = {}).rename(columns = {'affiliation_country_ed':'CoR', 'fractional_count_country_x': 'fract_count_CoR', 'econ_group_x':'econ_group_CoR', 'patentID':'patent_id', 'edited_company_country':'CoP', 'fractional_count_country_y':'fract_count_CoP', 'econ_group_y': 'econ_group_CoP'})

In [7]:
list(set(df_CoR_CoP['pubmed_id'][df_CoR_CoP['fract_count_CoP'] < 1]))

[1326592,
 11807235,
 10844680,
 27438093,
 9774606,
 23442959,
 12483598,
 15062547,
 11258900,
 7963157,
 12919318,
 8406037,
 1515027,
 12271129,
 22368282,
 8352795,
 8260635,
 19014685,
 10988064,
 7001633,
 11282978,
 11818531,
 10852900,
 9111074,
 12384294,
 8412707,
 15307817,
 15672873,
 10653739,
 16224815,
 14997552,
 16962608,
 2007602,
 2762291,
 15459382,
 10447926,
 7877689,
 3138109,
 9336897,
 11450947,
 15914052,
 16164420,
 10349636,
 15819847,
 12909128,
 25505353,
 18725962,
 8468553,
 9971796,
 9421909,
 8527447,
 3935325,
 15347807,
 20358240,
 16904802,
 12021349,
 12481130,
 9311851,
 2052603,
 6443128,
 16339577,
 14572666,
 11572857,
 9635450,
 10331265,
 8294017,
 8063108,
 14765194,
 21576845,
 15289485,
 8604305,
 1516690,
 10208916,
 8698519,
 8645272,
 7928984,
 1308825,
 19625627,
 17120926,
 22040740,
 19191973,
 11397798,
 9274025,
 16593578,
 1348781,
 26640049,
 12893874,
 12167859,
 9990324,
 22628533,
 14730428,
 10835645,
 26140862,
 15241406,
 

In [8]:
selection_list = list(set(df_CoR_CoP['pubmed_id'][df_CoR_CoP['fract_count_CoP'] < 1]))[:3]

In [9]:
0.5*0.33

0.165

In [30]:
df_CoR_CoP[df_CoR_CoP['pubmed_id'] == 8406037]

,pubmed_id,CoR,fract_count_CoR,econ_group_CoR,patent_id,CoP,fract_count_CoP,econ_group_CoP
5529,8406037,NL,0.5,OECD,118-929-448-073-446,US,1.0,OECD
5530,8406037,DE,0.5,OECD,118-929-448-073-446,US,1.0,OECD
6396,8406037,NL,0.5,OECD,001-770-161-687-382,US,1.0,OECD
6397,8406037,DE,0.5,OECD,001-770-161-687-382,US,1.0,OECD
15673,8406037,NL,0.5,OECD,185-037-247-220-653,US,1.0,OECD
15674,8406037,DE,0.5,OECD,185-037-247-220-653,US,1.0,OECD
15678,8406037,NL,0.5,OECD,042-408-116-309-900,US,1.0,OECD
15679,8406037,DE,0.5,OECD,042-408-116-309-900,US,1.0,OECD
15700,8406037,NL,0.5,OECD,143-164-459-416-090,US,0.5,OECD
15701,8406037,NL,0.5,OECD,143-164-459-416-090,KR,0.5,OECD


In [31]:
selection_list[:1]

[1326592]

In [27]:
edges_dict = {}

nr = 1
for pub in list(set(df_CoR_CoP['pubmed_id'])): 
    df_pub = df_CoR_CoP[df_CoR_CoP['pubmed_id'] == pub]
    edge_tuples = []
    
    for pat in list(set(df_pub['patent_id'])): 
        df_pat =  df_CoR_CoP[(df_CoR_CoP['pubmed_id'] == pub) &(df_CoR_CoP['patent_id'] == pat)]
        CoR_contribution = 1/len(set(df_pub['CoR'])) #calculate how much outflow there is for from that country for that publ
        CoP_use = 1/len(set(df_pat['CoP'])) # calculate how many patents resulted from that publ for that country
        for CoR in list(set(df_pub['CoR'])): 
            for CoP in list(set(df_pat['CoP'])): 
                    if (CoR,CoP) not in [x[:2] for x in edge_tuples]: 
                        edge_tuple = (CoR,CoP,CoR_contribution*CoP_use)
                        edge_tuples.append(edge_tuple)
                    else: 
                        list_element = 0
                        for single_tuple in edge_tuples:
                            
                            if (CoR,CoP) == single_tuple[:2]: 
                                edge_tuples[list_element] = (CoR, CoP,edge_tuples[list_element][2]+CoR_contribution*CoP_use)
                                #edge_tuples
                                
                                #single_tuple, single_tuple[:2]+(single_tuple[2]+CoR_contribution*CoP_use))
                            list_element = list_element +1
                        
    edges_dict[pub] = edge_tuples
    #nr = nr +1
    #if nr == 3: 
    #    break
        
        
        
        

In [17]:
pd.DataFrame(edge_tuples, columns = ['CoR','CoP','weights'])

,CoR,CoP,weights
0,JP,JP,5.0
1,JP,CH,1.0


In [29]:
edges_dict

{11239426: [('US', 'US', 7.0)],
 7987227: [('US', 'US', 0.5), ('DE', 'US', 0.5)],
 12435500: [('JP', 'US', 1.0)],
 21602357: [('KR', 'US', 3.0)],
 19923001: [('CA', 'CA', 1.0)],
 20807744: [('US', 'US', 3.0)],
 15466568: [('DE', 'US', 1.0)],
 12705876: [('JP', 'JP', 1.0)],
 18563158: [('US', 'AT', 0.2),
  ('GB', 'AT', 0.2),
  ('CH', 'AT', 0.2),
  ('JP', 'AT', 0.2),
  ('ES', 'AT', 0.2)],
 19136599: [('US', 'CL', 0.5), ('CA', 'CL', 0.5)],
 20152408: [('FR', 'FR', 1.0)],
 21160047: [('GB', 'DK', 0.5), ('ES', 'DK', 0.5)],
 8626290: [('GB', 'US', 5.0)],
 10322035: [('FR', 'DK', 1.0)],
 21512309: [('JP', 'US', 4.0)],
 7798902: [('US', 'US', 1.0)],
 16785531: [('GB', 'NL', 2.333333333333333),
  ('US', 'NL', 2.333333333333333),
  ('DE', 'NL', 2.333333333333333)],
 16769151: [('CN', 'CA', 1.0)],
 12869759: [('US', 'US', 1.0)],
 8224896: [('US', 'GB', 1.0)],
 23814277: [('CN', 'DE', 0.6666666666666666),
  ('GB', 'DE', 0.6666666666666666),
  ('US', 'DE', 0.6666666666666666)],
 8552584: [('US', 'U

In [32]:
edges_dict[8406037]

[('DE', 'KR', 1.0),
 ('DE', 'US', 3.0),
 ('NL', 'KR', 1.0),
 ('NL', 'US', 3.0),
 ('DE', 'JP', 0.5),
 ('NL', 'JP', 0.5)]

In [20]:
pub

15056888

In [14]:
final_edge_tuples = []
for keys,values in edges_dict.items():
    for single_tuple in values: 
        if single_tuple[:2] not in [x[:2] for x in final_edge_tuples]: 
            edge_tuple = (single_tuple[0],single_tuple[1],single_tuple[2])
            final_edge_tuples.append(edge_tuple)
        else: 
            list_element = 0
            for final_single_tuple in final_edge_tuples:
                if single_tuple[:2] == final_single_tuple[:2]: 
                    final_edge_tuples[list_element] = (final_single_tuple[0],final_single_tuple[1],final_single_tuple[2]+single_tuple[2])
                    #edge_tuples

                    #single_tuple, single_tuple[:2]+(single_tuple[2]+CoR_contribution*CoP_use))
                list_element = list_element +1

In [19]:
list_element

597

In [18]:
edge_tuple

('CL', 'GB', 0.5)

In [16]:
df = pd.DataFrame(final_edge_tuples, columns = ['CoR','CoP','weights'])

In [35]:
# add econ group
# load g77 and OECD data, assign overlapping to g77
g77 = pd.read_csv('PATH\data\g77.csv')
oecd = pd.read_csv('PATH\data\oecd.csv')

delete_from_oecd = []

for i in list(g77['Code']):
    if i in list(oecd['Code']):
        delete_from_oecd.append(i)

for i in list(g77['Code']):
    if i in list(oecd['Code']):
        delete_from_oecd.append(i)        

oecd_clean = [x for x in list(oecd['Code']) if x not in delete_from_oecd]

CoR_econ_group = []
for i in list(df['CoR']):
    if i in [pc.country_alpha3_to_country_alpha2(x) for x in oecd_clean]:
        CoR_econ_group.append('OECD')
    elif i in ['RU','BR','IN','CN','ZA']:
        CoR_econ_group.append('BRICS')
    elif i in [pc.country_alpha3_to_country_alpha2(x) for x in g77['Code'] if x not in ['RU','BR','IN','CN','ZA']]:
        CoR_econ_group.append('G77')
    elif pd.isna(i):
        CoR_econ_group.append(np.nan)
    else:
        CoR_econ_group.append('other')
df['CoR_econ_group'] = CoR_econ_group



CoP_econ_group = []
for i in list(df['CoP']):
    if i in [pc.country_alpha3_to_country_alpha2(x) for x in oecd_clean]:
        CoP_econ_group.append('OECD')
    elif i in ['RU','BR','IN','CN','ZA']:
        CoP_econ_group.append('BRICS')
    elif i in [pc.country_alpha3_to_country_alpha2(x) for x in g77['Code'] if x not in ['RU','BR','IN','CN','ZA']]:
        CoP_econ_group.append('G77')
    elif pd.isna(i):
        CoP_econ_group.append(np.nan)
    else:
        CoP_econ_group.append('other')
df['CoP_econ_group'] = CoP_econ_group

CoR_iso3 = []
for i in list(df['CoR']):
    CoR_iso3.append(pc.country_name_to_country_alpha3(pc.country_alpha2_to_country_name(i))) 
df['CoR_iso3'] = CoR_iso3

CoP_iso3 = []
for i in list(df['CoP']):
    CoP_iso3.append(pc.country_name_to_country_alpha3(pc.country_alpha2_to_country_name(i))) 
df['CoP_iso3'] = CoP_iso3

In [38]:
df.to_csv('PATH\data\CoR-CoP_fractional_edges_iso3.csv')